<a href="https://colab.research.google.com/github/Av01/Kinship-detection/blob/master/Kinship_detection_using_Siamese_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls

__notebook_source__.ipynb


In [0]:
mv kaggle.json ../root/.kaggle/kaggle.json

In [0]:
!chmod 600 ../root/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c recognizing-faces-in-the-wild

  0% 0.00/77.6k [00:00<?, ?B/s]
100% 77.6k/77.6k [00:00<00:00, 49.7MB/s]
 73% 25.0M/34.1M [00:00<00:00, 88.3MB/s]
100% 34.1M/34.1M [00:00<00:00, 86.2MB/s]
 92% 63.0M/68.6M [00:00<00:00, 94.9MB/s]
100% 68.6M/68.6M [00:00<00:00, 100MB/s] 
  0% 0.00/156k [00:00<?, ?B/s]
100% 156k/156k [00:00<00:00, 145MB/s]


In [0]:
from zipfile import ZipFile
with ZipFile('train.zip','r') as f:
  f.extractall()
with ZipFile('test.zip','r') as f:
  f.extractall()

In [0]:
!mkdir ../home/Model1 ../home/Model2

In [7]:
from google.colab import files

uploaded = files.upload()

!mv checkpoint ../home/Model1/checkpoint
!mv model.ckpt.data-00000-of-00001 ../home/Model1/model.ckpt.data-00000-of-00001
!mv model.ckpt.meta ../home/Model1/model.ckpt.meta
!mv model.ckpt.index ../home/Model1/model.ckpt.index

Saving checkpoint to checkpoint
Saving model.ckpt.data-00000-of-00001 to model.ckpt.data-00000-of-00001
Saving model.ckpt.index to model.ckpt.index
Saving model.ckpt.meta to model.ckpt.meta


In [9]:
uploaded = files.upload()
!mv checkpoint ../home/Model2/checkpoint
!mv model.ckpt.data-00000-of-00001 ../home/Model2/model.ckpt.data-00000-of-00001
!mv model.ckpt.meta ../home/Model2/model.ckpt.meta
!mv model.ckpt.index ../home/Model2/model.ckpt.index

Saving checkpoint to checkpoint
Saving model.ckpt.data-00000-of-00001 to model.ckpt.data-00000-of-00001
Saving model.ckpt.index to model.ckpt.index
Saving model.ckpt.meta to model.ckpt.meta


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path

import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
import glob
import os
import tensorflow as tf
%matplotlib inline

In [0]:
train_df = pd.read_csv("train_relationships.csv")
test_df = pd.read_csv("sample_submission.csv")

In [0]:
def process_image(image):
    normalized_img = np.mean(image,axis = 2,keepdims = True)/255
    shifted_img = np.roll(normalized_img,1,axis = 1)
    return normalized_img - shifted_img

In [0]:
import random
def generate_batch(pl1,pl2,show_size = False):
    p1 = np.copy(pl1)
    p2 = np.copy(pl2)
    positive_images = []
    for each_p1,each_p2 in zip(p1,p2):  #takes minimum of both
            images1 = [process_image(np.array(Image.open(f))) for f in glob.glob(each_p1 + "/*.jpg",recursive = True)]
            images2 = [process_image(np.array(Image.open(f))) for f in glob.glob(each_p2 + "/*.jpg",recursive = True)]
            entries = []
            for image1,image2 in zip(images1,images2):
                entry = [each_p1,each_p2,image1,image2,1]
                entries.append(entry)
            positive_images.extend(entries)
        
    negative_images = []
    random.shuffle(p2)
    for each_n1,each_n2 in zip(p1,p2):
            f1 = each_n1.split('/')[0]
            f2 = each_n2.split('/')[0]
            if f1 == f2:
#             print(f1,f2)
                continue
            images1 = [process_image(np.array(Image.open(f))) for f in glob.glob(each_n1 + "/*.jpg",recursive = True)]
            images2 = [process_image(np.array(Image.open(f))) for f in glob.glob(each_n2 + "/*.jpg",recursive = True)]
            entries = []
            for image1,image2 in zip(images1,images2):
                entry = [[each_n1,each_n2,image1,image2,0],[each_n2,each_n1,image2,image1,0]]
                entries.extend(entry)
            negative_images.extend(entries)
    pi = len(positive_images)
    ni = len(negative_images)
    if ni > pi:
        negative_images = random.sample(negative_images,pi) 
    if show_size:
        print("Sizes: ",len(positive_images),', ',len(negative_images))
    batch = negative_images + positive_images
    random.shuffle(batch)
    return batch

In [0]:
class Model:
    def __init__(self):
        self.keep_prob = tf.placeholder(dtype = tf.float32)
        self.rate = 1 - self.keep_prob 
        with tf.device('device:GPU:0'):
            self.X1 = tf.placeholder(dtype = tf.float32, shape = [None,224,224,1])
#             self.X1_n = tf.math.divide(self.X1,255)

            
        #weights
            self.w_c1 = tf.Variable(tf.random.normal([20,20,1,32],mean = 0, stddev = 0.01),name = 'conv_w1')
            self.w_c2 = tf.Variable(tf.random.normal([10,10,32,64],mean = 0, stddev = 0.01),name = 'conv_w2')
            self.w_c3 = tf.Variable(tf.random.normal([5,5,64,128],mean = 0, stddev = 0.01),name = 'conv_w3')

            self.w_l = tf.Variable(tf.random.normal([1000,1],mean = 0, stddev = 0.01),name = 'last_w')
            self.b_l = tf.Variable(tf.random.normal([1],mean = 0, stddev = 0.01),name = 'last_b')
    
        #left_side
        
            self.cl1 = tf.nn.conv2d(self.X1,self.w_c1,[1,2,2,1],"SAME")
            self.maxpooll1 = tf.nn.max_pool(self.cl1,ksize = [1,5,5,1],strides=[1,2,2,1],padding="SAME")
            self.dropoutl1 = tf.nn.dropout(self.maxpooll1,rate = self.rate)
            self.cl2 = tf.nn.conv2d(self.dropoutl1,self.w_c2,[1,2,2,1],"SAME")
            self.maxpooll2 = tf.nn.max_pool(self.cl2,ksize = [1,5,5,1],strides=[1,2,2,1],padding="SAME")
            self.dropoutl2 = tf.nn.dropout(self.maxpooll2,rate = self.rate) 
            self.cl3 = tf.nn.conv2d(self.dropoutl2,self.w_c3,[1,2,2,1],"SAME")
            self.maxpooll3 = tf.nn.max_pool(self.cl3,ksize = [1,5,5,1],strides=[1,2,2,1],padding="SAME")
            self.dropoutl3 = tf.nn.dropout(self.maxpooll3,rate = self.rate)
            d = self.dropoutl3.shape
        
            self.w_f = tf.Variable(tf.random.normal([int(d[1] * d[2] * d[3]), 1000],mean = 0, stddev = 0.01),name = 'fully_w')
            self.b_f = tf.Variable(tf.random.normal([1000],mean = 0, stddev = 0.01),name = 'fully_b')
        
            self.flatl1 = tf.reshape(self.dropoutl3, [-1,d[1] * d[2] * d[3]])
            self.l1 = tf.nn.relu(tf.matmul(self.flatl1,self.w_f) + self.b_f)
            print(self.l1.shape)

        
            self.l2 = tf.placeholder(dtype = tf.float32, shape = [None,1000])
            self.l = tf.math.abs(self.l1 - self.l2)
            self.out = tf.matmul(self.l,self.w_l) + self.b_l
            self.output = tf.math.sigmoid(100 * self.out)
            self.labels = tf.placeholder(dtype = tf.float32, shape = [None,1])
            self.loss = self.calculate_loss(self.output,self.labels)
            self.optimizer = self.optimize(self.loss)
            self.init = tf.global_variables_initializer()
        self.saver = tf.train.Saver([self.w_c1,self.w_c2,self.w_c3,self.w_f,self.b_f,self.w_l,self.b_l])
        
            
    def calculate_loss(self,out,label):
        return -tf.math.reduce_mean((label) * tf.math.log(out + 0.0001) + (1 - label) * tf.math.log(1 - out + 0.0001)) 
        
    def optimize(self,loss):
        return tf.train.RMSPropOptimizer(learning_rate = 0.0001).minimize(loss)
    
    def find_output(self,sess,X1,l2,keep_prob):
        output = sess.run(self.output,feed_dict = {self.keep_prob: keep_prob, self.X1 : X1, self.l2 : l2})
        return output
    
    def update_weights(self,sess,X1,l2,labels,keep_prob):
        loss,_ = sess.run([self.loss,self.optimizer],feed_dict = {self.keep_prob: keep_prob, self.X1 : X1, self.l2 : l2, self.labels : labels})
        return loss

In [0]:
def process_batch(batch):
    X1 = []
    X2 = []
    labels = []
    for each_entry in batch:
        X1.append(each_entry[2])
        X2.append(each_entry[3])
        labels.append([each_entry[4]])
    return np.array(X1),np.array(X2),np.array(labels)

In [0]:
import os
epoch = 100
batch_n = 32
train_data_n = train_df.shape[0]
train_n = int(0.98 * train_data_n)
test_n = train_data_n - train_n
train_df = train_df.sample(frac = 1)


In [63]:
tf.reset_default_graph()
model1 = Model()
model2 = Model()
Loss = []
with tf.Session() as sess:
    sess.run(model1.init)
    sess.run(model2.init)
    if os.path.exists("../home/Model1/checkpoint"):
        print("Restoring...")
        model1.saver.restore(sess,"../home/Model2/model.ckpt")
        model2.saver.restore(sess,"../home/Model1/model.ckpt")
    for e in range(epoch):
        print("Starting epoch: ", e + 1)
        Tloss = 0
        for i in range(0,train_n,batch_n):
            p1 = train_df['p1'][i:i + batch_n]
            p2 = train_df['p2'][i:i + batch_n]
            batch = generate_batch(p1,p2)
            X1,X2,labels = process_batch(batch)
            l2 = sess.run([model2.l1],feed_dict = {model2.keep_prob: 0.8, model2.X1 : X2})[0]
            output = model1.find_output(sess,X1,l2,0.8)
            loss = model1.update_weights(sess,X1,l2,labels,0.8)
#             out = sess.run([model1.out],feed_dict = {model1.X1 : X1, model1.l2 : l2})
#             print(out[:3])
#             output = sess.run([model.output],feed_dict = {model.X1 : X1, model.X2 : X2})
#             loss,_ = sess.run([model.loss,model.optimizer],feed_dict = {model.X1 : X1, model.X2 : X2, model.labels : labels})
#             print(sess.run([model.output[:10],model.w_c1[:2,:2,0,0]],feed_dict = {model.X1 : X1, model.X2 : X2, model.labels : labels}))
            Tloss += loss
            batch_accuracy = np.mean(np.equal(labels,np.round(output)))
            if i % (10 * batch_n) == 0:
                print("Batch accuracy: ", batch_accuracy)
                print("Batch loss: ",loss)
                model1.saver.save(sess,"../home/Model1/model.ckpt")
                model2.saver.save(sess,"../home/Model2/model.ckpt")
            model1,model2 = model2, model1
        print("Epoch ended")
        print("Total loss: ",Tloss)
        print("Average batch loss: ",(Tloss / (train_n//batch_n)))
        Loss.append(Tloss)

(?, 1000)
(?, 1000)
Restoring...
INFO:tensorflow:Restoring parameters from ../home/Model2/model.ckpt
INFO:tensorflow:Restoring parameters from ../home/Model1/model.ckpt
Starting epoch:  1
Batch accuracy:  0.8292682926829268
Batch loss:  0.3582245
Batch accuracy:  0.7564102564102564
Batch loss:  0.4533959
Batch accuracy:  0.8513513513513513
Batch loss:  0.34505785
Batch accuracy:  0.8412162162162162
Batch loss:  0.3940743
Batch accuracy:  0.8379629629629629
Batch loss:  0.40475935
Batch accuracy:  0.7978723404255319
Batch loss:  0.46287894
Batch accuracy:  0.8348214285714286
Batch loss:  0.41565832
Batch accuracy:  0.8252427184466019
Batch loss:  0.35722604
Batch accuracy:  0.7653061224489796
Batch loss:  0.46552214
Batch accuracy:  0.8227272727272728
Batch loss:  0.38691306
Batch accuracy:  0.7815126050420168
Batch loss:  0.42651317
Batch accuracy:  0.8217054263565892
Batch loss:  0.40495446
Epoch ended
Total loss:  43.33763003349304
Average batch loss:  0.39397845484993677
Starting ep

In [53]:
test_n

72

In [66]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = Model()
#     sess.run(model.init)
    output = []
    labels = []
    model.saver.restore(sess,"../home/Model2/model.ckpt")
    for i in range(train_n,train_data_n,batch_n):
        p1 = train_df['p1'][i:i+batch_n]
        p2 = train_df['p2'][i:i+batch_n]
        test_data = generate_batch(p1,p2)
        X1,X2,label = process_batch(test_data)
        l2 = sess.run([model.l1], feed_dict = {model.keep_prob: 1, model.X1 : X2})[0]
        out = model.find_output(sess,X1,l2,1)
        output.extend(out)
        labels.extend(label)
    accuracy = np.mean(np.equal(labels,np.round(output)))
    print("Testing Accuracy: ", accuracy)
    

(?, 1000)
INFO:tensorflow:Restoring parameters from ../home/Model2/model.ckpt
Testing Accuracy:  0.5543478260869565


In [28]:
#submission data
sub_data = pd.read_csv('sample_submission.csv')
sub_data_n = len(sub_data)
print(sub_data_n)
sub_data.head()


5310


,img_pair,is_related
0,face05508.jpg-face01210.jpg,0
1,face05750.jpg-face00898.jpg,0
2,face05820.jpg-face03938.jpg,0
3,face02104.jpg-face01172.jpg,0
4,face02428.jpg-face05611.jpg,0


In [0]:
def process_sub_data(files):
    X1 = []
    X2 = []
#     files = sub_data['img_pair']
    for entry in files:
#         print(entry)
        img1,img2 = entry.split('-')
        path = ''
        x1 = np.array(Image.open(path + img1))
        x2 = np.array(Image.open(path + img2))
        X1.append(process_image(x1))
        X2.append(process_image(x2))
#     print('Sizes: ',len(X1), ", ",len(X2))
    return np.array(X1),np.array(X2)

In [32]:
tf.reset_default_graph()
with tf.Session() as sess:
    model = Model()
    model.saver.restore(sess,'../home/Model1/model.ckpt')
    output = []
    files = sub_data['img_pair']
    for i in range(0,sub_data_n,10):
        X1,X2 = process_sub_data(files[i:i+10])
        l2 = sess.run([model.l1],feed_dict = {model2.keep_prob: 1, model.X1 : X2})[0]
        out = model.find_output(sess,X1,l2,1)
        output.extend(out)
    print(len(output))
    sub_data['is_related'] = np.round(np.array(output))

(?, 1000)
INFO:tensorflow:Restoring parameters from ../home/Model1/model.ckpt
5310


In [33]:
sub_data.tail()

,img_pair,is_related
5305,face99998.jpg-face99993.jpg,0.0
5306,face99997.jpg-face99996.jpg,0.0
5307,face99997.jpg-face99995.jpg,0.0
5308,face99997.jpg-face99994.jpg,0.0
5309,face99997.jpg-face99993.jpg,0.0


In [0]:
sub_data.to_csv('../home/submission.csv',index = False)

In [35]:
df = pd.read_csv('../home/submission.csv')
df.tail()

,img_pair,is_related
5305,face99998.jpg-face99993.jpg,0.0
5306,face99997.jpg-face99996.jpg,0.0
5307,face99997.jpg-face99995.jpg,0.0
5308,face99997.jpg-face99994.jpg,0.0
5309,face99997.jpg-face99993.jpg,0.0


In [36]:
files = ['../home/submission.csv']
files.extend(glob.glob('../home/Model?/*'))
print(files)


['../home/submission.csv', '../home/Model2/model.ckpt.meta', '../home/Model2/model.ckpt.index', '../home/Model2/model.ckpt.data-00000-of-00001', '../home/Model2/checkpoint', '../home/Model1/model.ckpt.meta', '../home/Model1/model.ckpt.index', '../home/Model1/model.ckpt.data-00000-of-00001', '../home/Model1/checkpoint']


In [0]:
image_folder = train_df['p1'][0]
image = np.array([np.array(Image.open(f)) for f in glob.glob("../input/train/" + image_folder + "/*.jpg", recursive = True)])
image.shape

In [0]:
normalized_img = np.mean(image,axis = 3)/255

In [0]:
import matplotlib.pyplot as plt
plt.imshow(normalized_img[1,:,:])


In [0]:
flattened_img = normalized_img.reshape(-1,224*224)
print(flattened_img.shape)
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(flattened_img)
pca

In [0]:
sum(pca.explained_variance_ratio_)

In [0]:
components = pca.components_
max_val = np.max( -1 * components,axis = 1)
max_mat = np.tile(np.array([max_val]),(components.shape[1],1)).T
components = (components + max_mat) / max_mat


In [0]:
fig, ax = plt.subplots(1,normalized_img.shape[0],figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(normalized_img[i])

In [0]:
fig, ax = plt.subplots(1,components.shape[0],figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(components[i].reshape(224,224))

In [0]:
def find_eigen_face(image_name):
    image = np.array([np.array(Image.open(f)) for f in glob.glob("../input/train/" + image_name + "/*.jpg", recursive = True)])
    print(image.shape)
    normalized_img = np.mean(image,axis = 3)/255 
    print(normalized_img.shape)
    flattened_img = normalized_img.reshape(-1,224*224) + 0.0001
    pca = PCA()
    pca.fit(flattened_img)
    components = pca.components_
    max_val = np.max( -1 * components,axis = 1)
    max_mat = np.tile(np.array([max_val]),(components.shape[1],1)).T
    components = (components + max_mat) / max_mat
    components = components.reshape(-1,224,224)
    return normalized_img, components

In [0]:
image_path = train_df['p1'][241]
norm,comp = find_eigen_face(image_path)

In [0]:
fig, ax = plt.subplots(1,min(norm.shape[0],8),figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(norm[i])

In [0]:
fig, ax = plt.subplots(1,min(norm.shape[0],8),figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(comp[i])

In [0]:
image_path = train_df['p2'][241]
print(image_path)
norm,comp = find_eigen_face(image_path)

In [0]:
fig, ax = plt.subplots(1,min(norm.shape[0],8),figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(norm[i])

In [0]:
fig, ax = plt.subplots(1,min(norm.shape[0],8),figsize = (40,40))
for i,axes in enumerate(ax.reshape(-1)):
    axes.imshow(comp[i])

In [0]:
i = 23

In [0]:
imgp = train_df['p1'][i]
img = [np.array(Image.open(f)) for f in glob.glob('../input/train/' + imgp + '/*.jpg', recursive = True)]
img1 = img[1]
print(img1.shape)
img1 = np.mean(img1,axis = 2)/255
image = img1 - np.roll(img1,1,axis = 1)


In [0]:
plt.imshow(image)

In [0]:
imgp = train_df['p2'][i]
img = [np.array(Image.open(f)) for f in glob.glob('../input/train/' + imgp + '/*.jpg', recursive = True)]
img1 = img[1]
print(img1.shape)
img1 = np.mean(img1,axis = 2)/255
image = img1 - np.roll(img1,1,axis = 1)

In [0]:
plt.imshow(image)